# 1D spectral analysis and light curve calculation from DL3 (energy-dependent angular cuts)

Originally created by Chaitanya Priyadarshi for the 1st LST-1 analysis school and later adapted to work with Gammapy v1.1 and the latest DL3 files produced by lstchain v0.10 for the 2nd LST-1 analysis school.

⚠️ **Important**: Gammapy v1.2 can also be used, but in this case LST-1 DL3 files must be produced with lstchain >= v0.10.9 to include the needed metadata to use Gammapy v1.2.

Even if Gammapy can handle gzipped DL3 files, it is faster to use unzipped DL3 files.

This is a combination of Gammapy tutorials for the **1D spectral analysis following the ON-OFF forward-folding method and light curve calculation** for point-like-source observations in wobble mode.
It requires point-like IRFs.

Original notebooks can be accessed at:

 - https://docs.gammapy.org/1.1/tutorials/analysis-1d/spectral_analysis_rad_max.html (from DL3 with energy-dependent angular cuts, the *approach followed in this notebook*)
 - https://docs.gammapy.org/1.1/tutorials/analysis-1d/spectral_analysis.html (from DL3 with global gammaness and angular cuts)
 - https://docs.gammapy.org/1.1/tutorials/analysis-time/light_curve.html
 - https://docs.gammapy.org/1.1/tutorials/analysis-time/light_curve_flare.html


It reduces a set of DL3 files (with energy-dependent angular cuts) into an energy-binned dataset, fits a spectral model to this dataset, calculates spectral flux points and computes the light curve.

We also describe how to write all these objects to files, read them back, inspect and plot the results.

-----------
## Content

### 0. Inspect the DL3 file content
### 1. Read the DL3 index files and load the data
### 2. Selection filters for the observations
### 3. Define Target position and energy ranges for reconstructed events
### 4. Define the base Map geometries for creating the SpectrumDataset
### 5. Data reduction chain
### 6. Generate the Spectrum Dataset for all observations
### 7. Some plots with the given Dataset
### 8. Write all datasets into OGIP files
### 9. Get Pivot energy to fix the reference energy and define the Spectrum Model
### 10. Spectral Fitting
### 11. Check the Flux points
### 12. SED plots
### 13. Light curve
### 14. Save the SED and LC Flux Points and Model to separate files

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import numpy as np
from regions import PointSkyRegion
from pathlib import Path
import os

In [ ]:
from gammapy.data import DataStore, EventList
from gammapy.datasets import (
    Datasets,
    FluxPointsDataset,
    SpectrumDataset,
)
from gammapy.estimators import FluxPointsEstimator, LightCurveEstimator, FluxPoints
from gammapy.makers import (
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    WobbleRegionsFinder
)
from gammapy.maps import MapAxis, RegionGeom, Map
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    Models,
    PowerLawSpectralModel,
    LogParabolaSpectralModel,
    create_crab_spectral_model,
    SkyModel,
)
from gammapy.visualization import plot_spectrum_datasets_off_regions

from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.time import Time
import astropy.units as u

from gammapy import __version__ as gammapy_version

In [ ]:
print(f"Using Gammapy {gammapy_version}")

# Input and output settings (SET BY THE USER)
Here we use some DL3 Crab data from the 2024 LST analysis school as an example.

If left unchanged, the following cell will create an analysis directory in your workspace. Please change it at your convenience.

Intermediate reduced datasets files are also stored following the OGIP standards.
See https://gamma-astro-data-formats.readthedocs.io/en/v0.3/spectra/ogip/index.html

In [ ]:
# Define the path to the input DL3 files.
base_dir = Path("/fefs/aswg/workspace/analysis-school-2024")
dl3_path = base_dir / "DL3/Crab"

# Define and create the analysis directory where products will be saved.
# It will be placed in your workspace, but you can change it accordingly.
username = os.getenv("USER")
analysis_dir = Path(f"/fefs/aswg/workspace/{username}/analysis-school-2024/high_level_analysis")

# Create also another directory to store the intermediate reduced dataset files in OGIP format.
ogip_path = analysis_dir / "OGIP"
ogip_path.mkdir(exist_ok=True, parents=True)  # All parent directories will be created.

# 0. Inspect the DL3 file content

In [ ]:
# Open one of the DL3 files from the directory
filename = dl3_path / "dl3_LST-1.Run15996.fits"
events = EventList.read(filename)

In [ ]:
# Have a look at the event list (here the first 10 events)
events.table[:10]

In [ ]:
# You can have a look of the events with
events.peek()

In [ ]:
# You can select events based any parameter on the table above, e.g. energy:
selected_energy = events.select_energy([500 * u.GeV, 1 * u.TeV])
selected_energy.peek()

In [ ]:
# or gammaness:
gh_range = [0.9, 1]
selected_events_gh = events.select_parameter(parameter="GAMMANESS", band=gh_range)
selected_events_gh.peek()

# 1. Read the DL3 index files and load the data

Let's load now all the DL3 files together.
If the DL3 index files are not present, run the `lstchain_create_dl3_index_files` for the given DL3 files.

`lstchain_create_dl3_index_files -d $dl3_path`  (by default create the index files in the same directory)

In [ ]:
total_datastore = DataStore.from_dir(dl3_path)

In [ ]:
# Let's have a first look at the first 5 entries of the table from all observations present in the index files. 
# It contains run-wise information.
total_datastore.obs_table[:5]

In [ ]:
# Also you can have a look at the HDU table, which contains information on the data 
# and instrument response function files for each observation
total_datastore.hdu_table[:6]

# 2. Selection filters for the observations

Based on the run-wise information from the previous table, filters can be applied to select just a subset of the observation list based on source name, zenith angle or livetime.

This list of observations based on similar filters can be also obtained beforehand by using the [data quality notebook](https://github.com/cta-observatory/cta-lstchain/blob/main/notebooks/data_quality.ipynb) (recommended). Then you can directly pass this list of `obs_id` to `get_observations` method of `DataStore` object below.

In [ ]:
# Get the object name from the OBS Table, assuming all the DL3 files are of the same single source.
# If not, then select a single object, to produce the relevant Spectrum Dataset

obj_name = np.unique(total_datastore.obs_table["OBJECT"])[0]
print("The source is", obj_name)

max_zen = 60  # in deg for a maximum limit on zenith pointing of observations
min_time = 60  # in seconds for minimum livetime of each observation

In [ ]:
total_obs_list = total_datastore.obs_table["OBS_ID"].data
observations_total = total_datastore.get_observations(
    total_obs_list, 
    required_irf="point-like",  # By default is "full-enclosure" : ["events", "gti", "aeff", "edisp", "psf", "bkg"]
    skip_missing=False  # Skip missing observations within the list provided earlier
)

In [ ]:
d_time = total_datastore.obs_table["LIVETIME"] > min_time
d_zen = total_datastore.obs_table["ZEN_PNT"] < max_zen
d_obj = total_datastore.obs_table["OBJECT"] == obj_name

obs_table_selected = total_datastore.obs_table[d_zen & d_obj & d_time]
obs_id_list = obs_table_selected["OBS_ID"]

observations_sel = total_datastore.get_observations(
    obs_id_list, 
    required_irf="point-like"
)

In [ ]:
print('Observation runs selected are:', obs_id_list.data)
print(f'Total livetime of all observations: {total_datastore.obs_table["LIVETIME"].to(u.h).sum():.1f}')
print(f'Total livetime of selected observations {obs_table_selected["LIVETIME"].data.sum()/3600:.1f} hrs')

# 3. Define Target position and energy ranges for reconstructed events

In [ ]:
target_position = SkyCoord.from_name(obj_name, frame='icrs')
target_position

If we were using global theta cut, this is the way of getting the theta cut (`RAD_MAX` key) used for the IRF production.

Here we will use energy-dependent cuts, therefore the following is not executed in this example.

```
# Find the fixed global theta cut used for creating the IRFs

theta_cut = observations_sel[0].aeff.meta["RAD_MAX"]
print("Theta cut applied for creating the IRF in the selected DL3 file,", theta_cut)

# Converting the value into astropy.units to be used for defining the ON region with CircleSkyRegion
on_region_radius = u.Quantity(theta_cut)
```

In [ ]:
# The metadata of the DL3 contains the efficiency of the cuts used for the DL3 production.
observations_sel[0].aeff.meta

In [ ]:
# Provide the minimum, maximum energies in TeV units, and number of bins per decade, to create the 
# required reconstructed and true energy ranges.
# For Light Curve estimation and spectral fitting, flux calculation can only be performed within 
# the energy edges provided for the reconstructed events.
# For example, if the reconstructed energy edges are [0.01, 0.1, 1, 10] TeV and you want LC in 
# [0.05, 10] TeV energy range, then, reproduce the Dataset objects with those reconstructed energy edges.

e_reco_min = 0.05  # 0.01
e_reco_max = 50

e_true_min = 0.01
e_true_max = 100

# Using bins per decade
e_reco_bin_p_dec = 8
e_true_bin_p_dec = 10

energy_axis = MapAxis.from_energy_bounds(
    e_reco_min, e_reco_max, 
    nbin=e_reco_bin_p_dec, per_decade=True, 
    unit="TeV", name="energy"
)
energy_axis_true = MapAxis.from_energy_bounds(
    e_true_min, e_true_max, 
    nbin=e_true_bin_p_dec, per_decade=True, 
    unit="TeV", name="energy_true"
)

# Select the minimum and maximum energy edges for the SED, from the energy_axis to be used in the Dataset
# Here we use a different minimum energy than energy_axis, but the same energy bins.
# For analyzers who do not want energy bins per decade, or some custom bins for energy_axis, 
# make appropriate changes in each axis.
e_fit_min = energy_axis.edges[1].value
e_fit_max = energy_axis.edges[-1].value
e_fit_bin_p_dec = e_reco_bin_p_dec

# Just to have a separate MapAxis for spectral fit energy range
energy_fit_edges = MapAxis.from_energy_bounds(
    e_fit_min, e_fit_max, 
    nbin=e_fit_bin_p_dec, per_decade=True, 
    unit="TeV"
).edges


print("Spectral Fit will be done in energy edges:\n", energy_fit_edges)

In [ ]:
# You can access the energy edges and center values
energy_axis_true.edges

# 4. Define the base Map geometries for creating the SpectrumDataset

In case of using global angular cut `CircleSkyRegion` should be used instead of `PointSkyRegion` (see ["Spectral analysis" Gammapy tutorial](https://docs.gammapy.org/1.2/tutorials/analysis-1d/spectral_analysis.html)).

In [ ]:
on_region = PointSkyRegion(target_position)  

# This will create the base geometry in which to bin the events based on their reconstructed positions
on_geom = RegionGeom.create(
    on_region, 
    axes=[energy_axis]
)

# 5. Data Reduction chain
Create some Dataset and Data Reduction Makers:

In the case of energy-dependent angular cuts we have to use the `WobbleRegionsFinder`, 
to determine the OFF positions, depending on the number of regions specified.

Their sizes will be defined by the theta values in RAD_MAX_2D table based on the estimated energy binning.
The same logic applies to the size of the ON region.

Background maker will use the WobbleRegionsFinder, assuming 1 OFF region for the background estimation
Using more off regions depends on the maximum value allowed for RAD_MAX_2D.
Gammapy will not allow the use of more off regions since at low energy they will overlap.

One way to overcome this problem is to restrict the value of theta angular cut up to a maximum value (e.g. 0.2 deg)
when producing the IRFs and DL3 files with lstchain.

In [ ]:
# geom is the target geometry in reco energy for counts and background maps
# energy_axis_true is the true energy axis for the IRF maps
dataset_empty = SpectrumDataset.create(
    geom=on_geom, 
    energy_axis_true=energy_axis_true
)
# When not including a PSF IRF, put the containment_correction as False
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, 
    selection=["counts", "exposure", "edisp"]
)

The following makers can be tuned to check the final Dataset to be used.

In [ ]:
region_finder = WobbleRegionsFinder(n_off_regions=1)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

In [ ]:
# Maker for safe energy range for the events, which can be based on effective area, energy bias, etc.
# Here we set a lower energy threshold corresponding to 5% of the maximum value of the effective area.
safe_mask_maker = SafeMaskMaker(
    methods=["aeff-max"], 
    aeff_percent=5
)

In [ ]:
# Alternatively you can make a custom safe energy range for the events.
#safe_min_energy = 50 * u.GeV
#safe_max_energy = 20 * u.TeV

For other arguments and options, check the SafeMaskMaker documentation:
https://docs.gammapy.org/1.1/api/gammapy.makers.SafeMaskMaker.html#gammapy.makers.SafeMaskMaker

# 6. Generate the Spectrum Dataset for all observations

In [ ]:
%%time
# The final object will be stored as a Datasets object

datasets = Datasets()

# Create a counts map for visualisation later
counts = Map.create(skydir=target_position, width=3)

for obs_id, observation in zip(obs_id_list, observations_sel):
    dataset = dataset_maker.run(
        dataset_empty.copy(name=str(obs_id)), 
        observation
    )
    dataset_on_off = bkg_maker.run(
        dataset=dataset, 
        observation=observation
    )
    counts.fill_events(observation.events)
    
    # Check the LC and SEDs by applying the safe mask to see the distinction.
    dataset_on_off = safe_mask_maker.run(dataset_on_off, observation)
    
    # Or use custom safe energy range
    #dataset_on_off.mask_safe = dataset_on_off.counts.geom.energy_mask(
    #    energy_min=safe_min_energy, energy_max=safe_max_energy
    #)
    
    datasets.append(dataset_on_off)    

Have a look at the information from the first dataset

In [ ]:
print(datasets[0])

## 6.1 Stack datasets

At this point, all the datasets can be stacked in a single dataset (SpectrumDatasetOnOff object), which is usually faster to handle.

You can do it like follows:

In [ ]:
stacked_dataset = Datasets(datasets).stack_reduce()

Alternatively, you can continue directly using the individual `Datasets` as produced in the above loop. Then you would perform a joint likelihood fit to all observations.

# 7. Some plots with the given Dataset

Plot the target position and center of the OFF regions used for the calculation of the gamma-ray excess.

In [ ]:
ax = counts.plot(cmap="viridis", stretch="sinh")
on_geom.plot_region(ax=ax, kwargs_point={"color": "k", "marker": "*"})
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets)
plt.show()

In [ ]:
info_table = datasets.info_table(cumulative=True)
info_table

In [ ]:
# Plot temporal evolution of excess events and significance value
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(
    info_table["livetime"].to("h"), info_table["excess"], marker="o", ls="none"
)
plt.plot(info_table["livetime"].to("h")[-1:1], info_table["excess"][-1:1], 'r')
plt.xlabel("Livetime (h)")
plt.ylabel("Excess")
plt.grid()
plt.title('Excess vs Livetime')

plt.subplot(122)
plt.plot(
    np.sqrt(info_table["livetime"].to("h")),
    info_table["sqrt_ts"],
    marker="o",
    ls="none",
)
plt.grid()
plt.xlabel(r"Sqrt Livetime ($\sqrt{h}$)")
plt.ylabel("sqrt_ts")
plt.title('Significance vs Square root of Livetime')
plt.subplots_adjust(wspace=0.5)


In [ ]:
%%time
# Plot the counts+excess, exposure and energy migration of each selected dataset.
# For simplicity we show them for the first two observations.
for data in datasets[:2]:
    plt.figure(figsize=(21, 5.5))
    plt.subplot(131)
    data.plot_counts()
    data.plot_excess()
    plt.grid(which="both")
    plt.title(f'Run {data.name} Counts and Excess')
    
    plt.subplot(132)
    data.exposure.plot()
    plt.grid(which='both')
    plt.title(f'Run {data.name} Exposure')
    
    plt.subplot(133)
    if data.edisp is not None:
        kernel = data.edisp.get_edisp_kernel()
        kernel.plot_matrix(add_cbar=True)
        plt.title(f'Run {data.name} Energy Dispersion')
    plt.subplots_adjust(wspace=0.3)
    plt.show()
    plt.close()

# 8. Write all datasets into OGIP files

These files are the datasets (events binned following the energy axes and geometry defined at the beginning). It corresponds to the DL4 data level.

In [ ]:
for dataset in datasets:
    dataset.write(
        filename=ogip_path / f"obs_{dataset.name}.fits.gz", overwrite=True
    )

In [ ]:
# Read the OGIP files to include the source object name in its headers, to be used for further analysis
for obs in obs_id_list:
    file = ogip_path/f"obs_{obs}.fits.gz"
    
    d1 = fits.open(file)
    d1.writeto(file, overwrite=True)

# 9. Get Pivot energy to fix the reference energy and define the Spectrum Model

As of Gammapy v1.2 it is possible to calculate the pivot energy for any spectral model not just for the power law like it is show below

In [ ]:
# Find pivot (decorrelation) assuming a Power Law model to get the reference energy for a Log Parabola model (Gammapy < v1.2)
def get_pivot_energy(datasets, e_ref, obj_name):
    """Calculate the decorrelation energy of the fit assuming a Power Law spectral model.
    
    This method (analytical calculation for a power law) is further explained 
    in doi:10.1088/0004-637X/707/2/1310
    """
    # As of Gammapy v1.2, the decorrelation energy can be calculated for any spectral model,
    # so you can change the model below to the one you use later.
    # For previous versions the calculation of the pivot energy was only available
    # (analytically) for the power law model.
    spectral_model = PowerLawSpectralModel(
        index=2, amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), reference=e_ref
    )

    model = SkyModel(spectral_model=spectral_model, name=obj_name)
    model_check = model.copy()

    # Stacked dataset method
    stacked_dataset = Datasets(datasets).stack_reduce()
    stacked_dataset.models = model_check

    fit_stacked = Fit()
    result_stacked = fit_stacked.run(datasets=stacked_dataset)

    return model_check.spectral_model.pivot_energy


In [ ]:
# Using a reference energy close to the expected decorrelation energy
ref = get_pivot_energy(datasets, 0.4 * u.TeV, obj_name)
print(f"Decorrelation energy used as reference energy: {ref.to(u.GeV):.1f}")

In [ ]:
# Define the spectral model to be fitted and later used to estimate the LC. 
# In this example, we assume a log parabola. One can try different Spectral Models as well.
# Be careful in the choice of Spectral Model being used for the 2 examples presented here

# Crab
spectral_model_lp = LogParabolaSpectralModel(
        amplitude = 5e-12 * u.Unit('cm-2 s-1 TeV-1'),
        reference = ref,
        alpha = 2 * u.Unit(''),
        beta = 0.1 * u.Unit('')
)

# Usually, setting minimum and maximum values for amplitude and assuming that 
# alpha and beta cannot be negative, (or the index in case of a simple power law),
# later helps to get ULs when using FluxPointEstimator in case of undetected sources
spectral_model_lp.amplitude.min = 1e-17
spectral_model_lp.amplitude.max = 1e-8

spectral_model_lp.alpha.min = 0
spectral_model_lp.beta.min = 0

model_lp = SkyModel(spectral_model=spectral_model_lp, name=obj_name)

In [ ]:
# Use the appropriate models, as per the selection of the source/dataset
model_lp.to_dict()['spectral']

# 10. Spectral Fitting
One can check for a more comprehensive tutorial on Modelling and Fitting in the Gammapy tutorials
* https://docs.gammapy.org/1.1/tutorials/api/fitting.html
* https://docs.gammapy.org/1.1/tutorials/api/model_management.html

In [ ]:
# We use the stacked analysis method, where we stack together all Datasets into 1 Dataset object and add the model afterward
stacked_dataset.models = model_lp

In [ ]:
# Fitting the model to the dataset
fit = Fit()
result = fit.run(datasets=stacked_dataset)
model_best = model_lp

In [ ]:
model_best.parameters[2]

In [ ]:
# Compute the Flux Points after Fitting the model
# We do not do too many optimizations here. 
# If one wants, can try and check the various attributes of the Estimator
fpe = FluxPointsEstimator(
    energy_edges=energy_fit_edges, 
    reoptimize = False, # Re-optimizing other free model parameters (not belonging to the source)
    source=obj_name,
    selection_optional="all" # Estimates asymmetric errors, upper limits and fit statistic profiles
)
flux_points = fpe.run(datasets=stacked_dataset)

flux_points_dataset = FluxPointsDataset(
    data=flux_points, models=model_best
)

In [ ]:
result

In [ ]:
model_best.to_dict()['spectral']['parameters']

# 11. Check the Flux points

In [ ]:
# Check the Flux table
# sed_type options are {“likelihood”, “dnde”, “e2dnde”, “flux”, “eflux”} with "likelihood" being default
# format options are {“gadf-sed”, “lightcurve”, “binned-time-series”, “profile”} with "gadf-sed" being default
flux_points.to_table(formatted=True, sed_type="e2dnde")

In [ ]:
# Fit Statistic array
print(flux_points_dataset.stat_array())

# Total statistics sum
print(flux_points_dataset.stat_sum(), np.nansum(flux_points_dataset.stat_array()))

In [ ]:
model_best.parameters.to_table()

# 12. SED Plots

In [ ]:
ref_label="Crab MAGIC LP (JHEAp 2015)"

In [ ]:
flux_points.to_table(formatted=True, sed_type="e2dnde")

In [ ]:
# Setting plot axes limits and other args

# Here we plot only in the energy range from where we have the first flux point, and before the first UL.
# This is based on the selection we used earlier on the dataset and the Flux points estimation we get.
# One should adjust the limit as per selections used earlier.

positive_flux = flux_points.to_table(formatted=True, sed_type="e2dnde")["e2dnde"] > 0
e_plot_min = flux_points.to_table(formatted=True, sed_type="e2dnde")["e_min"].quantity[positive_flux][0]

non_ul = flux_points.to_table(formatted=True, sed_type="e2dnde")["is_ul"] == 0
e_plot_max = flux_points.to_table(formatted=True, sed_type="e2dnde")["e_max"].quantity[non_ul][-1]

sed_kwargs = {
    "sed_type": "e2dnde",
    "energy_bounds": [e_plot_min, e_plot_max],
}

# Using the energy range used in the MAGIC reference
ds_magic_ref_kwargs = {
    "sed_type": "dnde",
    "energy_bounds": [50 * u.GeV, 30 * u.TeV],
}
sed_magic_ref_kwargs = {
    "sed_type": "e2dnde",
    "energy_bounds": [50 * u.GeV, 30 * u.TeV],
}
sed_plot_kwargs = {
    "label": "LST-1",
}
plot_ts_kwargs = {
    "color": "darkorange"
}

In [ ]:
# Calculate & plot Crab reference flux
# https://doi.org/10.1016/j.jheap.2015.01.002
crab = create_crab_spectral_model("magic_lp")
crab.amplitude.error = 0.03e-11 * u.Unit("cm-2 s-1 TeV-1")
crab.alpha.error = 0.01
crab.beta.error = 0.01/np.log(10)


In [ ]:
plt.figure(figsize=(8,5))
ax = flux_points.plot(sed_type="e2dnde", **plot_ts_kwargs)

flux_points.plot_ts_profiles(ax=ax, sed_type="e2dnde")
plt.xlim(e_plot_min.value, e_plot_max.value)

plt.grid(which='both')
plt.title('TS Profiles')

In [ ]:
# Fit model covariance matrix plot
model_best.covariance.plot_correlation()

In [ ]:
fig_sed = plt.figure(figsize=(8,8))

gs2 = GridSpec(7, 1)

gs2.update(hspace=0.1)
args1 = [gs2[:5,:]]
args2 = [gs2[5:,:]]

fig_gs1 = fig_sed.add_subplot(*args1)
fig_gs2 = fig_sed.add_subplot(*args2)

FluxPointsDataset(data=flux_points, models=model_best).plot_spectrum(
    ax=fig_gs1, 
    kwargs_fp=sed_plot_kwargs, 
)

create_crab_spectral_model("magic_lp").plot(
    ax=fig_gs1, **sed_magic_ref_kwargs, label=ref_label
)

fig_gs1.legend()
fig_gs1.set_xlim(e_plot_min.value, e_plot_max.value)
fig_gs1.set_ylim(2e-12, 2e-10)
fig_gs1.tick_params(labelbottom=False)

fig_gs1.grid(which='both')
fig_gs1.set_title('SED')

flux_points_dataset.plot_residuals(ax=fig_gs2, method='diff/model')
fig_gs2.grid(which='both')
fig_gs2.set_xlim(e_plot_min.value, e_plot_max.value);

In [ ]:
fig = plt.figure(figsize=(8,7))
gs = GridSpec(7, 1)

args1 = [gs[:5,:]]
args2 = [gs[5:,:]]
kwargs_res = {"method": "diff/sqrt(model)"}

fig_gs1 = fig.add_subplot(*args1)
fig_gs2 = fig.add_subplot(*args2)

stacked_dataset.plot_excess(fig_gs1)
fig_gs1.grid(which="both")
fig_gs1.set_ylabel("Excess")

stacked_dataset.plot_residuals_spectral(fig_gs2, **kwargs_res, region=stacked_dataset.counts.geom.region)
fig_gs2.grid(which="both")

fig_gs2.set_ylabel("Residuals \n (data-model)/sqrt(model)")

# 13. Light curve

In [ ]:
# Define the energy range for the light curve. It has to match the edges of the 
# reconstructed energy binning set for the SED! So if you want to calculate it between
# certain energy values, you must first define the energy axis accordingly.
e_lc_min = energy_axis.edges[4]
e_lc_max = energy_axis.edges[-1]
print(f"LC will be estimated from {e_lc_min.to(u.GeV):.0f} and {e_lc_max:.0f}")

In [ ]:
# Get the GTI parameters of each observation to create time intervals for plotting LC
t_start = []
t_stop = []
tot_time = []

for obs in observations_sel:
    gti = obs.gti
    
    t_start.append(gti.time_start[0])
    t_stop.append(gti.time_stop[0])
    tot_time.append(gti.time_sum.value)

t_start = np.sort(np.array(t_start))
t_stop = np.sort(np.array(t_stop))
tot_time = np.array(tot_time)

t_start = Time(t_start)
t_stop = Time(t_stop)

t_day = np.unique(np.rint(t_start.mjd))

# To make the range night-wise, keep the MJD range in half-integral values
t_range = [Time([t - 0.5, t + 0.5], format="mjd", scale="utc") for t in t_day]

In [ ]:
# Create the LC Estimator on a run-by-run basis and nightly
lc_maker_1d = LightCurveEstimator(
    energy_edges=[e_lc_min, e_lc_max], 
    reoptimize=False, # Re-optimizing other free model parameters (not belonging to the source)
    source=obj_name, 
    selection_optional="all" # Estimates asymmetric errors, upper limits and fit statistic profiles
)

lc_maker_night_wise = LightCurveEstimator(
    energy_edges=[e_lc_min, e_lc_max], 
    time_intervals=t_range,
    reoptimize=False, 
    source=obj_name,
    selection_optional="all"
)

# Assigning the best fit model for each dataset
for data in datasets:
    data.models = model_best

In [ ]:
lc_1d = lc_maker_1d.run(datasets)
lc_night = lc_maker_night_wise.run(datasets)

In [ ]:
# If there are more than 1 night of data, one can see the integrated light curve for each night
lc_night.to_table(sed_type="flux", format="lightcurve")

In [ ]:
# Check the various column data of the Light Curve object
lc_1d.to_table(sed_type="flux", format="lightcurve")

In [ ]:
# Calculate Crab reference flux
flux_crab, flux_crab_error = crab.integral_error(e_lc_min, e_lc_max)
print(f"Crab reference flux integrated between {e_lc_min.to(u.GeV):.0f} and {e_lc_max:.0f}: {flux_crab.value:.2e} +/- {flux_crab_error:.2e}")

In [ ]:
# Plotting settings
lc_kwargs = {
    "marker": "o",
}

fig_lc, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True, figsize=(8,10))
fig_lc.suptitle(
    f'Light curve for {obj_name}: {e_lc_min.to(u.GeV):.0f} < E < {e_lc_max.to(u.TeV):.0f} '
    f'\nRun-wise (top panel), night-wise (bottom panel), '
    f'{tot_time.sum()/3600:.1f} hrs, {len(t_day)} nights'
)

# Run-wise
lc_1d.plot(
    ax=ax1,
    sed_type="flux",
    label="LST-1 (run-wise)",
    **lc_kwargs
)
ax1.set_xlabel(None)

# Night-wise
lc_night.plot(
    ax=ax2,
    sed_type="flux",
    axis_name="time",
    label="LST-1 (night-wise)",
    **lc_kwargs
)

# Plot reference Crab Nebula flux
for ax in (ax1, ax2):
    ax.axhline(
        flux_crab.to_value("cm-2 s-1"), c='red', ls='--', 
        label=ref_label
    )
    ax.axhspan(
        (flux_crab - flux_crab_error).to_value("cm-2 s-1"), 
        (flux_crab + flux_crab_error).to_value("cm-2 s-1"), 
        alpha=0.2, color='tab:orange'
    ) 

ax2.set_yscale('linear')
ax2.set_ylim(0, 1.5 * np.nanmax(lc_1d.flux.data))
ax1.grid(which='both')
ax2.grid(which='both')
ax1.legend()
ax2.legend();

# 14. Save the SED and LC Flux Points and Model to separate files
This way, one can plot different SEDs and LCs together.

## Write spectral model to a file

By default, it also stores the covariance file together with the model (same file path as the model but adding the `_covariance.dat` suffix).

In [ ]:
spectral_model_file = analysis_dir / f'{obj_name}_dataset_{datasets[0].name}_to_{datasets[-1].name}_spectral_model.yml'
stacked_dataset.models.write(spectral_model_file, overwrite=True)

## Write the SED and LC flux points tables to disk

In [ ]:
flux_points_file = analysis_dir / f'{obj_name}_dataset_{datasets[0].name}_to_{datasets[-1].name}_SED_flux_points.fits'
lightcurve_file = analysis_dir / f'{obj_name}_dataset_{datasets[0].name}_to_{datasets[-1].name}_LC_flux_points.fits'

flux_points.write(flux_points_file, sed_type="e2dnde", format="gadf-sed", overwrite=True)
lc_1d.write(lightcurve_file, sed_type="flux", format="lightcurve", overwrite=True)

### Example on reading back and plotting the LC and SED

In [ ]:
flux_model = Models.read(spectral_model_file)

In [ ]:
flux_model

Then use the only model contained in the Models object we have created: `flux_model[0]`

In [ ]:
flux_points = FluxPoints.read(
    flux_points_file,
    format="gadf-sed",
    reference_model=flux_model[0]
)
flux_points_lc = FluxPoints.read(
    lightcurve_file,
    sed_type="flux", 
    format="lightcurve",
    reference_model=flux_model[0]
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.set_title(
    f'LC: {flux_points_lc.to_table(sed_type="flux", format="lightcurve")["e_min"].to(u.GeV)[0][0]:.0f} < '
    f'E < {flux_points_lc.to_table(sed_type="flux", format="lightcurve")["e_max"].to(u.TeV)[0][0]:.0f}'
)
flux_points_lc.plot(ax=ax1, sed_type="flux", label="LST-1 (run-wise)")

ax1.axhline(
    flux_crab.to_value("cm-2 s-1"), c='red', ls='--', label=ref_label
)
ax1.axhspan(
    (flux_crab - flux_crab_error).to_value("cm-2 s-1"), 
    (flux_crab + flux_crab_error).to_value("cm-2 s-1"), 
    alpha=0.2, color='tab:orange'
)
ax1.legend()
ax1.grid(which="both")

ax2.set_title("SED")
flux_points.plot(ax=ax2, sed_type="e2dnde", **sed_plot_kwargs)
flux_points.reference_model.spectral_model.plot(ax=ax2, **sed_kwargs)
flux_points.reference_model.spectral_model.plot_error(ax=ax2, **sed_kwargs)
create_crab_spectral_model("magic_lp").plot(
    ax=ax2,
    label=ref_label,
    **sed_magic_ref_kwargs,
)
ax2.legend()
ax2.grid()

# Exercise
Perform the 1D spectral analysis for BLLac with DL3 files in `/fefs/aswg/workspace/analysis-school-2024/DL3/BLLac` and calculate the light curve with a 10-minute temporal binning above 100 GeV.